<a href="https://colab.research.google.com/github/kimjh0107/Predict-solar-power-generation/blob/main/%EB%8F%99%EB%84%A4%EC%98%88%EB%B3%B4_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import urllib
import urllib.request
import json


In [2]:
ServiceKey = 'IG2r6UQ1wcIBT8CLCKuHgRKyMQw3pifU8133Tq2HMwK0c+aBcIMcFO691RQVxxmKlQzrjcwT3zYuYvpR7d3KAA=='
#'r' = Open a file for reading.
#decoding 인증키로 실행 

In [3]:
# 예보 받을 날짜를 입력해줘야 된다 
# urllib.parse.quote_plus('base_date')과 fcst_df = pd.DataFrame() | date = '2021-05-06' 부분 날짜를 서로 일치시켜줘야된다. 

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst'

queryParams = '?' + urllib.parse.urlencode(
    {
        urllib.parse.quote_plus('ServiceKey') : ServiceKey, # key를 바로 입력
        urllib.parse.quote_plus('numOfRows') : '113', # 총 14개의 항목을 3시간 단위로 순차적으로 불러옴 다음날 24시간예보
        urllib.parse.quote_plus('dataType') : 'JSON', # JSON, XML 두가지 포멧
        urllib.parse.quote_plus('base_date') : '20210506', # 예보 받을 날짜를 입력. 최근 1일간의 자료만 
        urllib.parse.quote_plus('base_time') : '1400', # 예보 시간을 입력. 2시부터 시작하여 3시간 단위로 입력 가능.
        urllib.parse.quote_plus('nx') : '102', # 울산 태양광 발전소 x 좌표 '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'에 포함 된 excel파일을 통해 확인 가능
        urllib.parse.quote_plus('ny') : '83' # 울산 태양광 발전소 y 좌표. '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'에 포함 된 excel파일을 통해 확인 가능
    }
)

response = urllib.request.urlopen(url + queryParams).read()
response = json.loads(response)

In [4]:
fcst_df = pd.DataFrame()
date = '2021-05-06'
fcst_df['Forecast_time'] = [f'{date} {hour}:00' for hour in range(24)]
row_idx = 0

In [5]:
fcst_df = pd.DataFrame()
date = '2021-05-06'
fcst_df['Forecast_time'] = [f'{date} {hour}:00' for hour in range(24)]
row_idx = 0

for i, data in enumerate(response['response']['body']['items']['item']):
  
    if i > 19:
        if data['category']=='REH':
            fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
            print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])

        elif data['category']=='T3H':
            fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
            print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])

        elif data['category']=='SKY':
            fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
            print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])

        elif data['category']=='VEC':
            fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
            print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])

        elif data['category']=='WSD':
            fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
            print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
            row_idx+=3

category:Humidity, REH baseTime: 1400 , fcstTime: 0000 , fcstValue: 45
category:Cloud, SKY baseTime: 1400 , fcstTime: 0000 , fcstValue: 3
category:Temperature, T3H baseTime: 1400 , fcstTime: 0000 , fcstValue: 16
category:WindDirection, VEC baseTime: 1400 , fcstTime: 0000 , fcstValue: 223
category:WindSpeed, WSD baseTime: 1400 , fcstTime: 0000 , fcstValue: 2.6 

category:Humidity, REH baseTime: 1400 , fcstTime: 0300 , fcstValue: 50
category:Cloud, SKY baseTime: 1400 , fcstTime: 0300 , fcstValue: 4
category:Temperature, T3H baseTime: 1400 , fcstTime: 0300 , fcstValue: 15
category:WindDirection, VEC baseTime: 1400 , fcstTime: 0300 , fcstValue: 217
category:WindSpeed, WSD baseTime: 1400 , fcstTime: 0300 , fcstValue: 3 

category:Humidity, REH baseTime: 1400 , fcstTime: 0600 , fcstValue: 65
category:Cloud, SKY baseTime: 1400 , fcstTime: 0600 , fcstValue: 3
category:Temperature, T3H baseTime: 1400 , fcstTime: 0600 , fcstValue: 15
category:WindDirection, VEC baseTime: 1400 , fcstTime: 0600 , 

In [6]:
# 예보를 한시간 단위로 선형보간 해주도록 한다 -> interpolate()

fcst_df = fcst_df.interpolate()
fcst_df = fcst_df.iloc[:24]

In [7]:
fcst_df

,Forecast_time,Humidity,Cloud,Temperature,WindDirection,WindSpeed
0,2021-05-06 0:00,45.000000,3.000000,16.000000,223.000000,2.600000
1,2021-05-06 1:00,46.666667,3.333333,15.666667,221.000000,2.733333
2,2021-05-06 2:00,48.333333,3.666667,15.333333,219.000000,2.866667
3,2021-05-06 3:00,50.000000,4.000000,15.000000,217.000000,3.000000
4,2021-05-06 4:00,55.000000,3.666667,15.000000,218.000000,3.166667
5,2021-05-06 5:00,60.000000,3.333333,15.000000,219.000000,3.333333
6,2021-05-06 6:00,65.000000,3.000000,15.000000,220.000000,3.500000
7,2021-05-06 7:00,60.000000,2.333333,16.666667,217.666667,4.166667
8,2021-05-06 8:00,55.000000,1.666667,18.333333,215.333333,4.833333
9,2021-05-06 9:00,50.000000,1.000000,20.000000,213.000000,5.500000
